In [ ]:
import custom_style; 
custom_style.style()

In [ ]:
import os
import sys
if os.sys.platform == 'darwin':
    sys.path.append('/Users/j35/git/BraggEdge/python')
else:
    sys.path.append(r"C:\Users\user\Documents\GitHub\BraggEdge\python")
    sys.path.append(r"C:\Users\user\Anaconda3\envs\SNAP2016\lib\site-packages")
 
import pandas as pd
import numpy as np
import matplotlib.patches as patches
import matplotlib.pyplot as plt
%matplotlib inline

from ipywidgets.widgets import interact
#from ipywidgets import widgets

from pprint import pprint

import datetime
import glob
import shutil
    
from neutronbraggedge.experiment_handler import *
from neutronbraggedge.braggedge import BraggEdge
from neutronbraggedge.material_handler.retrieve_material_metadata import RetrieveMaterialMetadata

from plotly.offline import plot, init_notebook_mode, iplot
init_notebook_mode()
import plotly.plotly as py
import plotly.graph_objs as go
from plotly import tools

from IPython.html import widgets
from IPython.display import display

import gui_widgets
import normalization
import load_handler

%gui qt

#### Load config

In [ ]:
from PyQt4.QtCore import QSettings
settings = QSettings()

default_dir = settings.value('default_dir')
if not(default_dir is None):
    default_dir = str(default_dir)
elif os.sys.platform == 'darwin':
    default_dir = r'/Users/j35/data/'
else:
    default_dir = r"C:\Data"

distance_source_detector = settings.value('distance_source_detector')
if not (distance_source_detector is None):
    distance_source_detector = float(distance_source_detector)
else:
    distance_source_detector = 15.89

detector_offset = settings.value('detector_offset')
if not (detector_offset is None):
    detector_offset = float(detector_offset)
else:
    detector_offset = 4200
    
list_of_elements = settings.value('list_of_elements')
if not (list_of_elements is None):
    list_of_elements = str(list_of_elements).split(',')
else:
    list_of_elements = ['Fe']
    
number_of_bragg_edges = settings.value('number_of_bragg_edges')
if not (number_of_bragg_edges is None):
    number_of_bragg_edges = int(number_of_bragg_edges)
else:
    number_of_bragg_edges = 8

## USER INPUT

#### Select a default starting directory (optional) 

In [ ]:
default_dir_widget = widgets.Text(
    description='default_dir',
    value=default_dir)
display(default_dir_widget)

#### What is the distance source-detector ?

In [ ]:
distance_source_detector_widget = widgets.Text(
    description = 'dSD (m)',
    value = str(distance_source_detector))
display(distance_source_detector_widget)

#### What is the detector offset ?

In [ ]:
detector_offset_widget = widgets.Text(
    description = 'detector_offset (micros)',
    value = str(detector_offset))
display(detector_offset_widget)

#### Full list of material available:

In [ ]:
retrieve_material = RetrieveMaterialMetadata(material = 'all')
list_returned = retrieve_material.full_list_material()
pprint(list_returned)

#### List of elements you want the Bragg Edges?     

ex: Li, Al, inconel

In [ ]:
list_of_elements_widget = widgets.Text(
    description = 'List elements',
    value = ','.join(list_of_elements))
display(list_of_elements_widget)

#### How many Bragg Edges do you want to see?


In [ ]:
number_of_bragg_edges_widget = widgets.Text(
    description = 'Nbr Bragg Edges',
    value = str(number_of_bragg_edges))
display(number_of_bragg_edges_widget)

#### Save config

In [ ]:
default_dir = default_dir_widget.value
distance_source_detector = float(distance_source_detector_widget.value)
detector_offset = float(detector_offset_widget.value)
list_of_elements = list_of_elements_widget.value.split(',')
number_of_bragg_edges = int(number_of_bragg_edges_widget.value)

settings.setValue('default_dir', str(default_dir))
settings.setValue('distance_source_detector', str(distance_source_detector))
settings.setValue('detector_offset', str(detector_offset))
settings.setValue('list_of_elements', ','.join(list_of_elements))
settings.setValue('number_of_bragg_edges', str(number_of_bragg_edges))

### Retrieve Powder Bragg Edges 

In [ ]:
_handler = BraggEdge(material=list_of_elements, 
                     number_of_bragg_edges = number_of_bragg_edges)
bragg_edges = _handler.bragg_edges
hkl = _handler.hkl

print(_handler)

## Loading Data

In [ ]:
%gui qt
list_files = gui_widgets.gui_fname(dir = default_dir)

# remove from list *_SummedImg.fits fille
list_files = [file for file in list_files if not "_SummedImg.fits" in file] 

# =================================
# Automatically load and convert Time Spectra
# =================================
dir_name = os.path.dirname(list_files[0])

spectra_files_name = glob.glob(dir_name + '/*_Spectra.txt')
if spectra_files_name == []:
    print("Select Spectra File...")
    spectra_file = gui_widgets.gui_single_file(dir = default_dir)
else:
    spectra_file = spectra_files_name[0]

_tof_handler = TOF(filename = spectra_file)

_exp = Experiment(tof = _tof_handler.tof_array, 
                  distance_source_detector_m = distance_source_detector, 
                  detector_offset_micros = detector_offset)

lambda_array = _exp.lambda_array * 1e10 # to be in Angstroms
tof_array = _exp.tof_array

# =================================
# Load Data
# =================================
w = widgets.IntProgress(descrption='Loading data')
w.max = len(list_files)
display(w)

data = []
for _index, _file in enumerate(list_files):
    _image = load_handler.load_data(_file)
    data.append(_image)
    w.value = _index + 1

### Crop Region

In [ ]:
# =================================
# Crop Data
# =================================
[height, width] = data[0].shape

def plot_image_for_cropping(file_index, x_left, x_right, y_top, y_bottom):
    
    fig = plt.figure(figsize=(10,10))
    ax_img = plt.subplot(111)
    ax_img.imshow(data[file_index])
    
#    _title = os.path.basename(list_files[file_index])
#    ax_img.set_title(_title)
    ax_img.set_title("Select Region To Crop", fontsize=15)
    ax_img.set_xlabel("Pixel x")
    ax_img.set_ylabel("Pixel Y")
 
    _rectangle = patches.Rectangle(
        (x_left, y_top),
        x_right - x_left,
        y_bottom - y_top,
        edgecolor = "white",
        linewidth = 2,
        fill = False)
    ax_img.add_patch(_rectangle)

    return [x_left, x_right, y_top, y_bottom]

number_of_files = len(list_files)
crop = interact(plot_image_for_cropping,
        file_index =widgets.IntSlider(min=0, 
                                      max=number_of_files-1,
                                      step=1, value=0, 
                                      description='Image index'),
        x_left = widgets.IntSlider(min=0, max=width-1, step=1, value=20),
        x_right = widgets.IntSlider(min=0, max=width-1, step=1, value=width-20),
        y_top = widgets.IntSlider(min=0, max=height-1, step=1, value=20), 
        y_bottom = widgets.IntSlider(min=0, max=height-1, step=1, value=height-20))

In [ ]:
# retrieve again powder bragg edges (in case number to display changed)
fig = tools.make_subplots(2, 1)

_handler = BraggEdge(material=list_of_elements, 
                     number_of_bragg_edges = number_of_bragg_edges)
bragg_edges = _handler.bragg_edges
hkl = _handler.hkl

[x_left, x_right, y_top, y_bottom] = crop.widget.result

sum_cropped_data = []
for _data in data:
    _sum_data = np.sum(_data[y_top:y_bottom, x_left:x_right])
    sum_cropped_data.append(_sum_data)
    
# format hkl labels
_hkl_formated = {}
for _material in hkl:
    _hkl_string = []
    for _hkl in hkl[_material]:
        _hkl_s = ",".join(str(x) for x in _hkl)
        _hkl_s = _material + "\n" + _hkl_s
        _hkl_string.append(_hkl_s)
    _hkl_formated[_material] = _hkl_string
    
trace = go.Scatter(
    x = lambda_array,
    y = sum_cropped_data,
    mode = 'markers')

trace2 = go.Scatter(
    x = tof_array,
    y = sum_cropped_data,
    mode = 'markers')

layout = go.Layout(
    width = "100%",
    height = 500,
    title = "Sum Counts vs TOF",
    xaxis = dict(
        title = "Lambda (Angstroms)"
                ),
    yaxis = dict(
        title = "Sum Counts"
                ),
    )

max_x = 6
y_off = 1

for y_index, _material in enumerate(bragg_edges):
    for _index, _value in enumerate(bragg_edges[_material]):
        if _value > max_x:
            continue
        bragg_line = {"type": "line",
                    'x0': _value,
                    'x1': _value,
                     'yref': "paper",
                     'y0': 0,
                     'y1': 1,
                     'line': {
                        'color': 'rgb(255, 0, 0)',
                        'width': 1
            }}
        layout.shapes.append(bragg_line)
        y_off = 1 - 0.25 * y_index
    
        # add labels to plots
        _annot = dict(
                    x=_value,
                    y= y_off,
                    text = _hkl_formated[_material][_index],
                    yref="paper",
                    font=dict(
                        family="Arial",
                        size=12,
                        color="rgb(150,50,50)"
                    ),
                    showarrow=True,
                    arrowhead=3,
                    ax=0,
                    ay=-25)
                
        layout.annotations.append(_annot)
        
#figure = tools.make_subplots(rows=2, cols=1)
#figure.append_trace(trace, 1, 1)
#figure.append_trace(trace2, 2, 1)
#figure.layout = layout

data1 = [trace]
def plot_final_image(x_left, x_right, y_top, y_bottom):
    layout.xaxis.range = [x_left, x_right]
    layout.yaxis.range = [y_bottom, y_top]
    figure = go.Figure(data=data1, layout=layout)
    iplot(figure)

resize = interact(plot_final_image,
        x_left = widgets.FloatSlider(min=lambda_array[0], max=lambda_array[-1], 
                                     description = 'Lambda min (Angstroms)',
                                     step=0.01, value=lambda_array[0]),
        x_right = widgets.FloatSlider(min=lambda_array[0], max=lambda_array[-1], 
                                      description = 'Lambda max (Angstroms)',
                                      step=0.01, value=lambda_array[-1]),
        y_top = widgets.FloatSlider(min=np.min(sum_cropped_data), max=max(sum_cropped_data),
                                        description = 'Max Counts',
                                       step=0.01, value=np.max(sum_cropped_data)),
        y_bottom = widgets.FloatSlider(min=np.min(sum_cropped_data), max=max(sum_cropped_data),
                                        description = 'Min Counts',
                                       step=0.01, value=np.min(sum_cropped_data)))
